
# Digging Analysis 

In this notebook we'll show examples of how to perform digging behavior analysis.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_gt = pd.read_excel('/content/timestamp.xlsx')

In [ ]:
df_gt.head()

In [ ]:
fps = 29.97

In [ ]:
box_x1, box_y1, box_x2, box_y2 = 324, 178, 570, 344

In [ ]:
def convert_time_to_frame_number(time_stamp, fps=29.97):
    h, m, s = time_stamp.split(':')
    seconds, milliseconds = s.split('.')
    total_seconds = int(h) * 3600 + int(m) * 60 + int(seconds)
    total_frames = int(total_seconds * fps) + int(milliseconds) * fps // 1000
    return int(total_frames)


In [ ]:
def convert_frame_number_to_time(fps, frame_number):
    total_seconds = frame_number / fps
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)
    milliseconds = int((total_seconds - int(total_seconds)) * 1000)
    time_stamp = f"{hours:02d}:{minutes:02d}:{seconds:02d}.{milliseconds:03d}"
    return time_stamp


In [ ]:
df_gt = df_gt['R2142_40-20-2022'].dropna().apply(convert_time_to_frame_number)

In [ ]:
df_gt.values

In [ ]:
df_pred = pd.read_csv('/content/rats_coco_dataset_R2142_04-20-2022_mask_rcnn_tracking_results_with_segmentation.csv')

In [ ]:
df_pred.head()

In [ ]:
df_pred.columns

In [ ]:
df_cup = df_pred[df_pred['instance_name']=='cup']

In [ ]:
df_cup.frame_number

In [ ]:
import ast
from pycocotools import mask as mask_util
def mask_area(mask):
    """Calulate the area of a RLE mask.
    """
    try:
        area = mask_util.area(mask)
    except TypeError:
        mask = ast.literal_eval(mask)
        area = mask_util.area(mask)
    return area

In [ ]:
df_cup['mask_area'] = df_cup.segmentation.apply(mask_area)

In [ ]:
df_cup.head()

In [ ]:
df_cup['mask_area'].describe()

In [ ]:
df_cup.class_score.describe()

In [ ]:
df_cup = df_cup[df_cup.mask_area > 400]

In [ ]:
df_cup = df_cup[df_cup.class_score > 0.88]

In [ ]:
df_cup.frame_number

In [ ]:
df_cups = df_pred[df_pred.frame_number.isin(df_cup.frame_number)]

In [ ]:
df_cups.frame_number

In [ ]:
df_cups_rats = df_cups[df_cups.frame_number.isin(df_cups[df_cups.instance_name == 'rat'].frame_number.unique())]

In [ ]:
df_cups_rats[df_cups_rats.frame_number == 5064]

In [ ]:
import pandas as pd

# Step 1: Filter dataframe to include frames with rat head and cup instances
rat_cup_df = df_cups_rats #df[(df['instance_name'] == 'rat head') | (df['instance_name'] == 'cup')]
# Step 2: Compute IoU for each frame
iou_list = []
for frame_num in rat_cup_df.frame_number:
    # Get binary masks for rat head and cup instances
    df_cur = rat_cup_df[rat_cup_df.frame_number == frame_num]
    rat_mask = df_cur[df_cur.instance_name == 'rat']['segmentation'].values[0]
    cup_mask = df_cur[df_cur.instance_name == 'cup']['segmentation'].values[0]
    rat_rle = ast.literal_eval(rat_mask)
    cup_rle = ast.literal_eval(cup_mask)
    # calculate the IoU between the rat and cup masks
    iou = mask_util.iou([rat_rle], [cup_rle], [0])[0][0]  # we assume there is only one rat and one cup mask
    iou_list.append((frame_num, iou))

# Convert results to dataframe
iou_df = pd.DataFrame(iou_list, columns=['frame_number', 'iou'])

In [ ]:
smoothed_iou = iou_df['iou'].rolling(15).mean()

In [ ]:
iou_df['smoothed_iou'] = smoothed_iou

In [ ]:
iou_df

In [ ]:
iou_df['diff_iou'] = iou_df.smoothed_iou.diff()


In [ ]:
# Compute rolling window of size 15 and count non-zero values
window_size = 15
nonzero_counts = iou_df['iou'].rolling(window_size).apply(lambda x: (x > 0).sum())


In [ ]:
iou_df['nonzero_counts'] = nonzero_counts

In [ ]:
iou_df.describe()

In [ ]:
prev_iou = 0.0
res = set()
for idx, row in iou_df.iterrows():
    if row.nonzero_counts >=1 and row.diff_iou > 0.000001 and  0 <= prev_iou <= 0.00000000000001:
        res.add(row.frame_number)
    prev_iou = row.diff_iou

In [ ]:
preds = [int(fn) for fn in sorted(res)]


In [ ]:
def reduce_array(arr):
    result = [arr[0]]  # initialize result array with first element of input array
    for i in range(1, len(arr)):
        if arr[i] - result[-1] < 400:
            result[-1] = min(result[-1], arr[i])
        else:
            result.append(arr[i])
    return result

In [ ]:
frames_cup_not_on_bottom = set(df_cup[(df_cup.cy < box_y1) | (df_cup.cy > box_y2) | (df_cup.cx < box_x1) | (df_cup.cx > box_x2)].frame_number.unique())


In [ ]:
preds = [pred for pred in preds if pred not in frames_cup_not_on_bottom]

In [ ]:
df_cup[df_cup.frame_number == 142073]

In [ ]:
preds = reduce_array(preds)

In [ ]:
len(preds), print(str(preds))

In [ ]:
len(df_gt.values), print(str(df_gt.values))

In [ ]:
time_stamps = []
for pred in preds:
    time_stamps.append(convert_frame_number_to_time(fps,pred))


In [ ]:
time_stamps

In [ ]:
pred_s = pd.Series(time_stamps)

In [ ]:
pred_s.to_csv('predicted_R2142_04-20-2022.csv', index=False)

In [ ]:
# create the plot
fig, ax = plt.subplots(figsize=(16, 8))

# plot the data
ax.plot(iou_df["frame_number"], iou_df["smoothed_iou"], label="Smoothed IoU")
ax.plot(iou_df["frame_number"], iou_df["diff_iou"], label="Difference of IoU")

# plot the special marks
x_gt = df_gt.values
y_gt = [ax.get_ylim()[1]] * len(x_gt)
ax.scatter(x_gt, y_gt, marker="|", color="red", s=150, label="Ground Truth")
x_pred = preds
y_pred = [ax.get_ylim()[1]] * len(x_pred)
ax.scatter(x_pred, y_pred, marker='|',color='green', s=150, label="Predictions")

# add legend
ax.legend(loc="lower left")

# show the plot
plt.show()